In [78]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

# fix random seed for reproducibility
np.random.seed(42)

In [79]:
%%time
# Load training set
X = pd.read_hdf("data/gliomablastomaTrain.h5", "expression")
Y = pd.read_hdf("data/gliomablastomaTrain.h5", "labels")

CPU times: user 404 ms, sys: 216 ms, total: 620 ms
Wall time: 612 ms


In [80]:
X.head()

,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,snoU2-30,snoU2_19,snoU83B,snoZ196,snoZ278,snoZ40,snoZ6,snosnR66,uc_338,yR211F11.2
THR14_0297_S01,0.0,0.0,0.000000,3.234195,1.929791,0.000000,5.994127,0.464668,1.879706,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304835,0.0
THR14_0301_S01,0.0,0.0,0.000000,5.514122,4.040893,0.014355,8.977251,1.124328,0.201634,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218609,0.0
THR14_0302_S01,0.0,0.0,0.000000,3.559492,1.169925,0.014355,8.616255,2.229588,0.475085,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.481127,0.0
THR14_0304_S01,0.0,0.0,0.049631,5.558880,2.662205,0.014355,10.800576,1.903038,0.028569,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.193246,0.0
THR14_0305_S01,0.0,0.0,0.056584,4.239551,2.220330,0.000000,8.710049,1.137504,0.042644,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200588,0.0


In [81]:
X.describe()

,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,snoU2-30,snoU2_19,snoU83B,snoZ196,snoZ278,snoZ40,snoZ6,snosnR66,uc_338,yR211F11.2
count,225.000000,225.0,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,...,225.0,225.0,225.0,225.0,225.0,225.0,225.0,225.0,225.000000,225.000000
mean,0.185784,0.0,0.164601,4.244867,2.093188,0.011011,7.614568,0.917209,1.069062,0.002997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.690336,0.023499
std,0.504135,0.0,0.360338,1.566839,0.873243,0.022809,1.628085,0.574407,0.937392,0.014871,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.147353,0.049487
min,0.000000,0.0,0.000000,0.137504,0.137504,0.000000,0.014355,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
25%,0.000000,0.0,0.000000,2.980032,1.443607,0.000000,7.069670,0.526069,0.298782,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.089355,0.000000
50%,0.000000,0.0,0.000000,4.537348,2.260073,0.000000,8.011497,0.799151,0.739848,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.193246,0.000000
75%,0.000000,0.0,0.075006,5.429636,2.739831,0.014355,8.593138,1.163561,1.713763,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.438614,0.000000
max,3.272023,0.0,2.169925,7.999743,4.136664,0.137632,10.800576,4.048759,3.486771,0.124328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.953731,0.286881


In [82]:
Y.head()

,th_sampleid,tissue,disease,TR_method
180,THR14_0297_S01,"hippocampus,temporal lobe",glioblastoma multiforme,PolyA
187,THR14_0301_S01,temporal lobe,glioblastoma multiforme,PolyA
188,THR14_0302_S01,thalamus,glioblastoma multiforme,PolyA
190,THR14_0304_S01,parietal lobe,glioblastoma multiforme,PolyA
191,THR14_0305_S01,"frontal lobe,parietal lobe,cerebellum/posterio...",glioblastoma multiforme,PolyA


In [83]:
# Convert tumor_normal and primary_site into numerical values for two-hot multi-class training
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y["TR_method_value"] = pd.Series(encoder.fit_transform(Y["TR_method"]), index=Y.index)
encoder = LabelEncoder()
Y["tissue_value"] = pd.Series(encoder.fit_transform(Y["tissue"]), index=Y.index)
Y.describe(include="all", percentiles=[])

,th_sampleid,tissue,disease,TR_method,TR_method_value,tissue_value
count,225,225,225,225,225.000000,225.000000
unique,225,10,1,2,NaN,NaN
top,TCGA-28-5215-01,brain,glioblastoma multiforme,PolyA,NaN,NaN
freq,1,166,225,184,NaN,NaN
mean,NaN,NaN,NaN,NaN,0.182222,3.075556
std,NaN,NaN,NaN,NaN,0.386888,1.277755
min,NaN,NaN,NaN,NaN,0.000000,0.000000
50%,NaN,NaN,NaN,NaN,0.000000,3.000000
max,NaN,NaN,NaN,NaN,1.000000,9.000000


In [84]:
X

,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,snoU2-30,snoU2_19,snoU83B,snoZ196,snoZ278,snoZ40,snoZ6,snosnR66,uc_338,yR211F11.2
THR14_0297_S01,0.000000,0.0,0.000000,3.234195,1.929791,0.000000,5.994127,0.464668,1.879706,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304835,0.000000
THR14_0301_S01,0.000000,0.0,0.000000,5.514122,4.040893,0.014355,8.977251,1.124328,0.201634,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218609,0.000000
THR14_0302_S01,0.000000,0.0,0.000000,3.559492,1.169925,0.014355,8.616255,2.229588,0.475085,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.481127,0.000000
THR14_0304_S01,0.000000,0.0,0.049631,5.558880,2.662205,0.014355,10.800576,1.903038,0.028569,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.193246,0.000000
THR14_0305_S01,0.000000,0.0,0.056584,4.239551,2.220330,0.000000,8.710049,1.137504,0.042644,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200588,0.000000
THR14_0306_S01,0.000000,0.0,0.063503,2.160275,0.505891,0.000000,7.371994,1.678072,0.137504,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.393538,0.000000
THR14_0308_S01,0.000000,0.0,0.090853,5.176722,3.314697,0.070389,8.917879,3.139142,0.536053,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.297028,0.000000
THR14_0310_S01,0.000000,0.0,0.115477,1.526069,0.505891,0.000000,8.080444,1.622930,0.650765,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010780,0.000000
THR14_0311_S01,0.000000,0.0,0.000000,5.242983,2.448901,0.000000,8.718259,1.843984,0.070389,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065419,0.056584
THR14_0316_S01,0.000000,0.0,0.000000,5.178715,2.488001,0.000000,8.436836,2.304511,2.007195,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.141142,0.000000


In [105]:
Y

,th_sampleid,tissue,disease,TR_method,TR_method_value,tissue_value
180,THR14_0297_S01,"hippocampus,temporal lobe",glioblastoma multiforme,PolyA,0,6
187,THR14_0301_S01,temporal lobe,glioblastoma multiforme,PolyA,0,8
188,THR14_0302_S01,thalamus,glioblastoma multiforme,PolyA,0,9
190,THR14_0304_S01,parietal lobe,glioblastoma multiforme,PolyA,0,7
191,THR14_0305_S01,"frontal lobe,parietal lobe,cerebellum/posterio...",glioblastoma multiforme,PolyA,0,5
192,THR14_0306_S01,thalamus,glioblastoma multiforme,PolyA,0,9
194,THR14_0308_S01,thalamus,glioblastoma multiforme,PolyA,0,9
196,THR14_0310_S01,thalamus,glioblastoma multiforme,PolyA,0,9
197,THR14_0311_S01,thalamus,glioblastoma multiforme,PolyA,0,9
202,THR14_0316_S01,frontal lobe,glioblastoma multiforme,PolyA,0,4


In [103]:
# gene expression
y = np.array((Y["TR_method"]=='RiboMinus').astype(np.int)) # 1 if RiboMinus, else 0

In [104]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [106]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X, y)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [76]:
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
print(X_new)

[[ 0.        ]
 [ 0.003003  ]
 [ 0.00600601]
 [ 0.00900901]
 [ 0.01201201]
 [ 0.01501502]
 [ 0.01801802]
 [ 0.02102102]
 [ 0.02402402]
 [ 0.02702703]
 [ 0.03003003]
 [ 0.03303303]
 [ 0.03603604]
 [ 0.03903904]
 [ 0.04204204]
 [ 0.04504505]
 [ 0.04804805]
 [ 0.05105105]
 [ 0.05405405]
 [ 0.05705706]
 [ 0.06006006]
 [ 0.06306306]
 [ 0.06606607]
 [ 0.06906907]
 [ 0.07207207]
 [ 0.07507508]
 [ 0.07807808]
 [ 0.08108108]
 [ 0.08408408]
 [ 0.08708709]
 [ 0.09009009]
 [ 0.09309309]
 [ 0.0960961 ]
 [ 0.0990991 ]
 [ 0.1021021 ]
 [ 0.10510511]
 [ 0.10810811]
 [ 0.11111111]
 [ 0.11411411]
 [ 0.11711712]
 [ 0.12012012]
 [ 0.12312312]
 [ 0.12612613]
 [ 0.12912913]
 [ 0.13213213]
 [ 0.13513514]
 [ 0.13813814]
 [ 0.14114114]
 [ 0.14414414]
 [ 0.14714715]
 [ 0.15015015]
 [ 0.15315315]
 [ 0.15615616]
 [ 0.15915916]
 [ 0.16216216]
 [ 0.16516517]
 [ 0.16816817]
 [ 0.17117117]
 [ 0.17417417]
 [ 0.17717718]
 [ 0.18018018]
 [ 0.18318318]
 [ 0.18618619]
 [ 0.18918919]
 [ 0.19219219]
 [ 0.1951952 ]
 [ 0.19819

In [74]:

y_proba = log_reg.predict_proba(X_new)
plt.plot(X_new, y_proba[:, 1], "g-", label="RiboMinus")
plt.plot(X_new, y_proba[:, 0], "b--", label="Not RiboMinus")
# + more Matplotlib code to make the image look pretty

ValueError: X has 1 features per sample; expecting 58581